## Heaps

In [4]:
class Heap(object):
    def __init__(self):
        self.heaplist = []
        
    def __len__(self):
        return len(self.heaplist)
    
    def __getitem__(self, idx):
        return self.heaplist[idx]
    
    def add(self, item):
        self.heaplist.append(item)
        self._heapify_up(len(self.heaplist) - 1)
    
    def _heapify_up(self, idx):
        if self.heaplist[idx] < self.heaplist[idx//2]:
            self.heaplist[idx], self.heaplist[idx//2] = self.heaplist[idx//2], self.heaplist[idx]
            self._heapify_up(idx//2)
        
    def _get_min_child(self, idx):
        if ((idx * 2) + 2) > len(self.heaplist) - 1:
            return (idx * 2) + 1
        else:
            if self.heaplist[(idx * 2) + 1] < self.heaplist[(idx * 2) + 2]:
                return (idx * 2) + 1
            return (idx * 2) + 2
    
    def _heapify_down(self, idx):
        min_child = self._get_min_child(idx)
        if self.heaplist[idx] > self.heaplist[min_child]:
            self.heaplist[idx], self.heaplist[min_child] = self.heaplist[min_child], self.heaplist[idx]
            if not min_child*2 + 1 > len(self.heaplist) - 1:
                self._heapify_down(min_child)
    
    def del_min(self):
        last_element = self.heaplist.pop()
        self.heaplist[0] = last_element
        self._heapify_down(0)
        

In [5]:
def _is_min_heap(L, i):
    l, r = 2 * i + 1, 2 * i + 2

    if r < len(L): # has left and right children
        if L[l] < L[i] or L[r] < L[i]: # heap property is violated
            return False

        # check both children trees
        return _is_min_heap(L, l) and _is_min_heap(L, r)
    elif l < len(L): # only has left children
        if L[l] < L[i]: # heap property is violated
            return False

        # check left children tree
        return _is_min_heap(L, l)
    else: # has no children
        return True

print(_is_min_heap([-1, 1, 3, 5, 7], 0))
print(_is_min_heap([1, 5, 3, 7], 0))

True
True


In [6]:
a = Heap()
a.add(3)
print(_is_min_heap(a, 0))
a.add(5)
print(_is_min_heap(a, 0))
a.add(7)
print(_is_min_heap(a, 0))
a.add(1)
print(_is_min_heap(a, 0))
a.add(-1)
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)

True
True
True
True
True
[-1, 1, 3, 5, 7]
True
[1, 5, 3, 7]
True
[3, 5, 7]
True
[5, 7]


#### Merge k sorted lists
* Create a min heap of size k. Initialise with first element of each array
* Repeat while min heap is not empty
    * Pop min and add an element from the same array as the popped element
    * Add element to results
    * If there are no more elements from that array, do nothing

In [2]:
def merge_k_sorted_arrays(sorted_arrs):
    return []

a = [0, 3, 6, 9]
b = [2, 4, 5, 7]
c = [1, 8, 10]

print(merge_k_sorted_arrays([a, b, c]))

[]


#### Sort an increasing decreasing array
* Reverse the decreasing pieces of the array
* Now we have k sorted arrays that need merging
    * Use a heap of size k
    * get elements form array where it was popped from and add to heap
    * if no new elements exists do nothing
    
#### Sort an almost sorted array
* Each number is atmost k distance away
* After the first k + 1 numbers have been read, the smallest number in the group is the smallest number
    * maintain a rolling heap of size k + 1
    * keep popping and adding to result and move to end like a sliding window
    
#### K closest planets to earth
* Each planet is a certain distance away
    * maintain max heap of size k
    * Keep adding elements, evict elements that are greater than the result
    * maintain a rolling heap of size k + 1

#### Implement stack using heap
* We care about the most recently inserted element
    * This element should always be at the top
    * So attach a timestamp that can be used as the comparator
    * Use a max heap that captures the element with highest timestamp